In [35]:
! pip install -U scikit-learn scipy matplotlib

In [36]:
path = '/home/anurag'
#****************************To be Change to reproduce ou result*********************************************

# location of the data and where to store iamge feature image
path_output_chd = path+'/Med_VQA'    

input_vqa_train = 'train_dataset_pickle/train19_subset20_21_df.pkl'
input_vqa_valid ='valid_dataset_pickle/val19_df.pkl'

img_feat_train = 'train_dataset_pickle/train-image-feature-19.pickle'
img_feat_valid ='valid_dataset_pickle/valid-image-feature-19.pickle'

input_test = 'test_dataset_pickle/test19_df.pkl'
img_feat_test = 'test_dataset_pickle/test-image-feature-19.pickle'

In [37]:
import json
with open(path+'/Med_VQA/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)


l = len(answer_classes) 

print(l)

178


In [38]:
import easydict
opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 64,
        "VAL_BATCH_SIZE": 64,
        "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 1472,
        "INIT_LERARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":100,
    })

In [39]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import re
import collections
import pickle
import os
from tqdm import tqdm
import json
import torch.nn as nn
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision
from PIL import Image
import torch.nn.functional as f
import random
from sklearn.preprocessing import LabelEncoder

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [41]:
path_change = path
seed_value = opt.SEED
print(seed_value) # 97
np.random.seed(seed_value) # return None
random.seed(seed_value) # return None
torch.manual_seed(seed_value) # return <torch._C.Generator object at 0x7f71cdf7a3d0>
torch.cuda.manual_seed(seed_value) # return None
torch.cuda.manual_seed_all(seed_value) # return None
torch.backends.cudnn.enabled = False 
''' backends.cudnn.enabled enables cudnn for some operations such as conv layers and RNNs, which can yield 
a significant speedup. The cudnn RNN implementation doesn’t support the backward operation during eval() 
and thus raises the error. You could disable cudnn for your workload (as already done) or try to call .train()
on the RNN module separately after using model.eval().'''
torch.backends.cudnn.benchmark = False
'''If your model does not change and your input sizes remain the same - then you may benefit from setting 
torch.backends.cudnn.benchmark = True.However, if your model changes: for instance, if you have layers that
are only "activated" when certain conditions are met, or you have layers inside a loop that can be iterated a 
different number of times, then setting torch.backends.cudnn.benchmark = True might stall your execution.'''
torch.backends.cudnn.deterministic = False
'''torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. 
Therefore, no, it will not guarantee that your training process is deterministic, since you're also using
torch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA.'''

97


"torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. \nTherefore, no, it will not guarantee that your training process is deterministic, since you're also using\ntorch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA."

In [42]:
#set path for the dataset
train_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/'#QAPairsByCategory/'
train_images_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/Train_images/'

valid_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/'#QAPairsByCategory/'
valid_images_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/Val_images/'

test_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/' 
test_images_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/VQAMed2019_Test_Images/'

size = opt.IMG_INPUT_SIZE
size

224

In [43]:
#Extract image feature
class VGG19(nn.Module):
    def __init__(self):
        '''
             We remove all the fully-connected layers in the VGG19 network and the convolution outputs of different feature scales
                are concatenated after global average pooling and l2-norm to form a 1984-dimensional vector to represent the image
        '''
        super(VGG19,self).__init__()
        vgg_model = torchvision.models.vgg19(pretrained=True)	
        # resnet_model = torchvision.models.resnet(pretrained=True)

        self.Conv1 = nn.Sequential(*list(vgg_model.features.children())[0:4])
        self.Conv2 = nn.Sequential(*list(vgg_model.features.children())[4:9])
        self.Conv3 = nn.Sequential(*list(vgg_model.features.children())[9:16])
        self.Conv4 = nn.Sequential(*list(vgg_model.features.children())[16:23])
        self.Conv5 = nn.Sequential(*list(vgg_model.features.children())[23:30])
        self.Conv6 = nn.Sequential(*list(vgg_model.features.children())[30:36])

        self.avgpool = nn.Sequential(list(vgg_model.children())[1])

        # self.inc = nn.Linear(1984,2048)
    
    def forward(self,image):

        with torch.no_grad():
            out1 = self.Conv1(image)
            # print('out1 shape',out1.shape)
            out2 = self.Conv2(out1)
            out3 = self.Conv3(out2)
            out4 = self.Conv4(out3)          
            out5 = self.Conv5(out4)          # [N, 512, 14, 14]
            out6 = self.Conv6(out5) 
            out7 = self.avgpool(out6)

        #global average pooling
        out1 = out1.mean([2,3],keepdim=True)
        out2 = out2.mean([2,3],keepdim=True)
        out3 = out3.mean([2,3],keepdim=True)
        out4 = out4.mean([2,3],keepdim=True)
        out5 = out5.mean([2,3],keepdim=True)
        out6 = out6.mean([2,3],keepdim=True)
        out7 = out7.mean([2,3],keepdim=True)

       
         
        concat_features = torch.cat([out1,out2, out3, out4,out5,out6 ], 1) 
        # print('concat_features',concat_features.shape) # 1,1984,1,1
        # with torch.no_grad():
        #     concat_features = self.inc(torch.squeeze(concat_features))
        
        # concat_features = torch.unsqueeze(concat_features, 0)
        # concat_features = torch.unsqueeze(concat_features, 2)
        # concat_features = torch.unsqueeze(concat_features, 3)

        #l2-normalized feature vector
        l2_norm = concat_features.norm(p=2, dim=1, keepdim=True).detach() 
        concat_features = concat_features.div(l2_norm)               # l2-normalized feature vector
       

        batch_size = concat_features.shape[0]
        embedding_dim_size = concat_features.shape[1]
        image_feature = concat_features.view(batch_size, embedding_dim_size, -1).squeeze(0) # [N, 1984, 1]

        # print('image_feature shape',image_feature.shape)

        return image_feature

In [44]:
vgg19_model = VGG19().to(device)
# vgg19_model = ResNet50().to(device)

/home/anurag/anaconda3/envs/med_vqa/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/anurag/anaconda3/envs/med_vqa/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [45]:
! pip install torchinfo

In [46]:
from torchinfo import summary
print(summary(vgg19_model))

Layer (type:depth-idx)                   Param #
VGG19                                    --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       1,792
│    └─ReLU: 2-2                         --
│    └─Conv2d: 2-3                       36,928
│    └─ReLU: 2-4                         --
├─Sequential: 1-2                        --
│    └─MaxPool2d: 2-5                    --
│    └─Conv2d: 2-6                       73,856
│    └─ReLU: 2-7                         --
│    └─Conv2d: 2-8                       147,584
│    └─ReLU: 2-9                         --
├─Sequential: 1-3                        --
│    └─MaxPool2d: 2-10                   --
│    └─Conv2d: 2-11                      295,168
│    └─ReLU: 2-12                        --
│    └─Conv2d: 2-13                      590,080
│    └─ReLU: 2-14                        --
│    └─Conv2d: 2-15                      590,080
│    └─ReLU: 2-16                        --
├─Sequential: 1-4                       

In [47]:
transform = {
    phase: transforms.Compose([transforms.RandomResizedCrop(size), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))]) 

    for phase in ['train', 'valid']}

test_transform = transforms.Compose([transforms.RandomResizedCrop(size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])

In [48]:
def parse_sentence(s):
    '''remove some character on the question'''
    s = s.replace(" - ", "-")
    s = s.lower()
    s = re.sub("\s\s+", " ", s)
    return s

In [49]:
train_dataset_df = np.load('/home/anurag/Med_VQA/train_dataset_pickle/train19_subset20_21_df.pkl',allow_pickle=True)

In [50]:
train_dataset_df.head(5)

,ID,Q,A,PATH,labels
0,synpic41148,what kind of image is this?,cta - ct angiography,/home/anurag/Med_VQA/data/raw/ImageCLEF/train_...,32
1,synpic43984,is this a t1 weighted image?,no,/home/anurag/Med_VQA/data/raw/ImageCLEF/train_...,2
2,synpic38930,what type of imaging modality is used to acqui...,us - ultrasound,/home/anurag/Med_VQA/data/raw/ImageCLEF/train_...,15
3,synpic52143,is this a noncontrast mri?,no,/home/anurag/Med_VQA/data/raw/ImageCLEF/train_...,2
4,synpic20934,what type of image modality is this?,xr - plain film,/home/anurag/Med_VQA/data/raw/ImageCLEF/train_...,5


In [51]:
image_feat_train={}

In [52]:
for row in train_dataset_df.itertuples():
    # print(row)
    image = Image.open(row.PATH).convert('RGB')
    image = transform['train'](image)
    # print(type(image))
    image_feature = vgg19_model(image[None,...].to(device))
    image_feat_train[row.ID] = image_feature.cpu().numpy()

In [53]:
valid_dataset_df = np.load('/home/anurag/Med_VQA/valid_dataset_pickle/val19_df.pkl',allow_pickle=True)

In [54]:
valid_image_feat={}

In [55]:
for row in valid_dataset_df.itertuples():
    # print(row)
    image = Image.open(row.PATH).convert('RGB')
    image = transform['valid'](image)
    # print(type(image))
    image_feature = vgg19_model(image[None,...].to(device))
    valid_image_feat[row.ID] = image_feature.cpu().numpy()

In [56]:
class VqaDataset(data.Dataset):
    '''
        Main class use to retrieve our dataset from pickle file.
    '''

    def __init__(self, input_vqa, img_feat_vqa, transform=None, phase = 'train'):
        # self.input_dir = input_dir
        # self.vqa = np.load(input_dir+'/'+input_vqa, allow_pickle=True )
        # self.img_feat_vqa = np.load(input_dir+'/'+img_feat_vqa, allow_pickle=True )
        self.vqa = input_vqa
        self.vocab_size = None
        self.phase = phase
        self.transform = transform
   

    def __getitem__(self, idx):

        vqa = self.vqa
        phase = self.phase
        transform=self.transform[phase]
        # img_feat_vqa = self.img_feat_vqa
        
        # image_id = vqa['image_id'].values[idx]
        image_id = vqa['ID'].values[idx]
        # image_path = vqa['image_path'].values[idx]
        image_path = vqa['PATH'].values[idx]
        # image_feat = torch.Tensor(img_feat_vqa[image_id])
        # print('image_path',image_path)
        
        image = Image.open(image_path).convert('RGB')
        # print('transform',transform)
        if transform:
            image = transform(image)

        qst2idc =  vqa['Q'].values[idx]
        sample = { 'question': qst2idc, 'image': image, 'image_id':image_id, 'image_path': image_path } 
        if (self.phase == 'train') or  (self.phase == 'valid'):
            ans2idc = vqa['labels'].values[idx]
            answer_text = vqa['A'].values[idx]
            sample['image_id'] = image_id
            sample['label'] = ans2idc
            sample['answer_text'] = answer_text
        else:
            sample['image_id'] = image_id
            
            
        return sample

    def __len__(self):

        return len(self.vqa)


def get_loader(input_vqa_train, input_vqa_valid, batch_size, num_workers, transform=None,size=228):
    '''
        Load our dataset with dataloader for the train and valid data
    '''

    vqa_dataset = {
        'train': VqaDataset(
            # input_dir=input_dir,
            input_vqa=input_vqa_train,
            img_feat_vqa=img_feat_train,
            transform = transform,
            phase = 'train'),
        'valid': VqaDataset(
            # input_dir=input_dir,
            input_vqa=input_vqa_valid,
            img_feat_vqa=img_feat_valid,
            transform = transform,
            phase = 'valid')}
    

    data_loader = {
        phase: torch.utils.data.DataLoader(
            dataset=vqa_dataset[phase],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            )
        for phase in ['train','valid']}

    return data_loader


def get_test_loader(input_test, batch_size, num_workers,size=228):

   
    
    test_vqa_dataset = VqaDataset(
            # input_dir=input_dir,
            input_vqa=input_test,
            # img_feat_vqa=img_feat_vqa,
            phase = 'test')
    
    
    data_loader = torch.utils.data.DataLoader(dataset=test_vqa_dataset,
                                                transform=transform,
                                                batch_size=batch_size,
                                                shuffle=False,
                                                num_workers=num_workers)
    return data_loader

In [57]:
input_vqa_train = train_dataset_df
input_vqa_valid = valid_dataset_df

saved_dir = path_output_chd

num_epochs = opt.NUM_EPOCHS
image_size = opt.IMG_INPUT_SIZE
num_workers = 0
batch_size = opt.BATCH_SIZE

In [58]:
data_loader = get_loader(
        input_vqa_train = train_dataset_df, 
        input_vqa_valid = input_vqa_valid,
        batch_size = batch_size, 
        num_workers = num_workers,
        transform = transform,
        size = image_size )

In [59]:
def mixup_data_vqa(id,v,p, q, a, al, alpha=1.0, use_cuda=True):
    '''Returns mixed images, pairs of questions, targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = v.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    # print('index',index)

    mixed_v = lam * v + (1 - lam) * v[index, :]
    # print('mixed_v',mixed_v.shape)
    id_a, id_b = id,id[index]
    p_a,p_b = p,p[index]
    a_a, a_b = a, a[index]
    al_a,al_b = al,al[index]
    # print('a_a',a_a)
    # print('a_b',a_b)
    q_a, q_b = q, q[index]
    # print('q_a',q_a)
    # print('q_b',q_b)
    
    return id_a,id_b,mixed_v,p_a,p_b, a_a, a_b,al_a,al_b, q_a, q_b, lam

In [60]:
image_feat = {}
dict_data={'image_id': [],  'image_path_a': [], 'image_path_b':[] ,'question_a': [], 'question_b':[], 'labels_a':[] , 'labels_b':[], 'answer_a' : [], 'answer_b':[], 'lamda':[]  }

In [61]:
for batch_idx, batch_sample in enumerate(data_loader['train']):
    image_id = batch_sample['image_id']
    image_path = batch_sample['image_path']
    image = batch_sample['image'].to(device)
    questions = batch_sample['question']
    labels = batch_sample['label'].to(device)
    label_answer_text = batch_sample['answer_text']
    le1 = LabelEncoder()
    label_answers_text_l = le1.fit_transform(label_answer_text)
    label_answers_text_l = torch.as_tensor(label_answers_text_l).to(device)
    le2 = LabelEncoder() 
    question_labels = le2.fit_transform(questions)
    question_labels = torch.as_tensor(question_labels).to(device)
    le3 = LabelEncoder()
    image_id_labels = le3.fit_transform(image_id)
    image_id_labels = torch.as_tensor(image_id_labels).to(device)
    le4 = LabelEncoder()
    image_path_labels = le4.fit_transform(image_path)
    image_path_labels = torch.as_tensor(image_path_labels).to(device)

    id_a,id_b,mixed_v, p_a,p_b,a_a, a_b,al_a,al_b, q_a, q_b, lam = mixup_data_vqa(
                image_id_labels,image, image_path_labels,question_labels, labels, label_answers_text_l, alpha=0.1, use_cuda=True)

    # mixed_v = mixed_v.cpu().data.numpy()
    a_a = a_a.cpu().data.numpy()
    a_b = a_b.cpu().data.numpy()
    id_a = id_a.cpu().data.numpy()
    id_b = id_b.cpu().data.numpy()
    q_a = q_a.cpu().data.numpy()
    q_b = q_b.cpu().data.numpy()
    al_a = al_a.cpu().data.numpy() 
    al_b = al_b.cpu().data.numpy()
    p_a = p_a.cpu().data.numpy()
    p_b = p_b.cpu().data.numpy()
    p_a_l = le4.inverse_transform(p_a)
    p_b_l = le4.inverse_transform(p_b)
    id_a_l = le3.inverse_transform(id_a)
    id_b_l = le3.inverse_transform(id_b)
    q_a_l = le2.inverse_transform(q_a)
    q_b_l = le2.inverse_transform(q_b)
    al_a_l = le1.inverse_transform(al_a)
    al_b_l = le1.inverse_transform(al_b)
    


    for index,img in enumerate(mixed_v):
        # print('img_shape',img.shape)
        mix_image_feature = vgg19_model(img[None,...]).to(device)
        # print('mix_img_feature.shape',mix_image_feature.shape)
        image_feat[id_a_l[index]+'_'+id_b_l[index]] = mix_image_feature.cpu().numpy()
        image_feat_train[id_a_l[index]+'_'+id_b_l[index]] = mix_image_feature.cpu().numpy()

    # imshow(torchvision.utils.make_grid(mixed_v))
    mixed_v = mixed_v.cpu().data.numpy()
    
    for index in range(len(mixed_v)):
        # print(id_a[index])
        image_name = id_a_l[index]+'_'+id_b_l[index]
        dict_data['image_id'].append(image_name)
        # dict_data['image_id'].append(id_a_l[index]+'_'+id_b_l[index])
        # dict_data['mixed_image'].append(mixed_v[index])
        # dict_data['mixed_image'].append(mixed_v[index])
        dict_data['image_path_a'].append(p_a_l[index])
        dict_data['image_path_b'].append(p_b_l[index])
        dict_data['question_a'].append(q_a_l[index])
        dict_data['question_b'].append(q_b_l[index])
        dict_data['labels_a'].append(a_a[index])
        dict_data['labels_b'].append(a_b[index])
        dict_data['answer_a'].append(al_a_l[index])
        dict_data['answer_b'].append(al_b_l[index])
        
        dict_data['lamda'].append(lam)
        # mixed_image = np.transpose(mixed_v[index], (1, 2, 0))
        # print(mixed_image.min())
        # mini = mixed_image.min()
        # maxi = mixed_image.max()
        # print(mixed_image.max())
        # print(mini)
        # print(maxi)
        # plt.imshow(mixed_image)
        # new_mixed_image = (1/(mini*(-1)*maxi)) * mixed_image + 0.5
        # new_mixed_image = mixed_image/(maxi-mini) + 0.5
        # plt.imsave('myy.png',(mixed_image).astype(np.uint8),cmap='gray')
        # fig, axs = plt.subplots(1, 3, figsize=(9, 3))
        # axs[0].imshow(Image.open(p_a_l[index]).convert('RGB'))
        # axs[0].set_title(id_a_l[index])
        # axs[1].imshow(Image.open(p_b_l[index]).convert('RGB'))
        # axs[1].set_title(id_b_l[index])
        # axs[2].imshow(mixed_image)
        # axs[2].set_title(image_name)
        # # plt.show()
        # fig.savefig(path+'/Med_VQA/data/raw/mixed/'+image_name+'.png',bbox_inches='tight',pad_inches=1)

        # plt.close()

In [62]:
train_dict = train_dataset_df.to_dict('list')
print('train_dict',train_dict.keys())

train_dict dict_keys(['ID', 'Q', 'A', 'PATH', 'labels'])


In [63]:
for key in dict_data:
    if key == 'image_id':
        for i in train_dict['ID']:
            dict_data[key].append(i)
    elif key == 'image_path_a':
        for i in train_dict['PATH']:
            dict_data[key].append(i)
    elif key == 'image_path_b':
        for i in train_dict['PATH']:
            dict_data[key].append(i)
    elif key == 'question_a':
        for i in train_dict['Q']:
            dict_data[key].append(i)
    elif key == 'question_b':
        for i in train_dict['Q']:
            dict_data[key].append(i)
    elif key == 'answer_a':
        for i in train_dict['A']:
            dict_data[key].append(i)
    elif key == 'answer_b':
        for i in train_dict['A']:
            dict_data[key].append(i)
    elif key == 'labels_a':
        for i in train_dict['labels']:
            dict_data[key].append(i)
    elif key == 'labels_b':
        for i in train_dict['labels']:
            dict_data[key].append(i)
    elif key == 'lamda':
        for i in train_dict['Q']:
            dict_data[key].append(1.0)

In [64]:
mixed_train_df_data = pd.DataFrame(dict_data, columns = ['image_id',
                                                    'mixed_image', 
                                                    'question_a',
                                                    'question_b',
                                                    'answer_a',
                                                    'answer_b',
                                                    'labels_a',
                                                    'labels_b',
                                                    'lamda'
                                                ])

In [65]:
mixed_train_df_data


,image_id,mixed_image,question_a,question_b,answer_a,answer_b,labels_a,labels_b,lamda
0,synpic26679_synpic17696,NaN,which organ system is imaged?,what is abnormal in the ct scan?,vascular and lymphatic,sarcoidosis,18,66,0.185757
1,synpic52640_synpic19015,NaN,what imaging modality is used to acquire this ...,what imaging method was used?,xr - plain film,xr - plain film,5,5,0.185757
2,synpic46480_synpic26947,NaN,is this a ct scan?,which image modality is this?,no,nm - nuclear medicine,2,47,0.185757
3,synpic54051_synpic54829,NaN,what is the primary abnormality in this image?,what organ system is shown in the image?,leptomeningeal sarcoid,musculoskeletal,177,6,0.185757
4,synpic48337_synpic41703,NaN,what part of the body does this mri show?,what is the primary abnormality in this image?,skull and contents,plantar fasciitis,1,177,0.185757
...,...,...,...,...,...,...,...,...,...
28427,synpic44352,NaN,what is abnormal in the mri?,what is abnormal in the mri?,neuroblastoma,neuroblastoma,105,105,1.000000
28428,synpic34315,NaN,what abnormality is seen in the image?,what abnormality is seen in the image?,adenocarcinoma of the lung,adenocarcinoma of the lung,71,71,1.000000
28429,synpic57872,NaN,what is abnormal in the angiogram?,what is abnormal in the angiogram?,"dural fistula, avf","dural fistula, avf",107,107,1.000000
28430,synpic17802,NaN,what is abnormal in the ct scan?,what is abnormal in the ct scan?,osteoid osteoma,osteoid osteoma,113,113,1.000000


In [66]:
mixed_train_df_data.shape

(28432, 9)

In [67]:
pickle_name='mixed_train_dataset_df_19_subset2021'
mixed_train_df_data.to_pickle(path_output_chd+'/train_dataset_pickle/'+pickle_name+'.pkl')

In [68]:
with open(path_output_chd+'/train_dataset_pickle/mixed-and-original-train-image-feature-19-subset2021.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(image_feat_train, f, pickle.HIGHEST_PROTOCOL)

In [73]:
v = np.load('/home/anurag/Med_VQA/test_dataset_pickle/test19_df.pkl',allow_pickle=True)

In [74]:
v

,ID,Task,Q,A,PATH
0,synpic54082,modality,what modality is shown?,cta - ct angiography,/home/anurag/Med_VQA/data/raw/ImageCLEF/test_2...
1,synpic48556,modality,was the mri taken with contrast?,no,/home/anurag/Med_VQA/data/raw/ImageCLEF/test_2...
2,synpic50696,modality,what type of contrast did this patient have?,iv,/home/anurag/Med_VQA/data/raw/ImageCLEF/test_2...
3,synpic37194,modality,what imaging method was used?,us-d - doppler ultrasound,/home/anurag/Med_VQA/data/raw/ImageCLEF/test_2...
4,synpic31308,modality,what modality is shown?,an - angiogram,/home/anurag/Med_VQA/data/raw/ImageCLEF/test_2...
...,...,...,...,...,...
495,synpic21789,abnormality,what is abnormal in the ct scan?,ossification of stylohyoid ligament,/home/anurag/Med_VQA/data/raw/ImageCLEF/test_2...
496,synpic39878,abnormality,what is most alarming about this mri?,cerebral infarct,/home/anurag/Med_VQA/data/raw/ImageCLEF/test_2...
497,synpic41525,abnormality,what is most alarming about this ct scan?,epiploic appendagitis,/home/anurag/Med_VQA/data/raw/ImageCLEF/test_2...
498,synpic18173,abnormality,what is the primary abnormality in this image?,pancreatic ductal adenocarcinoma,/home/anurag/Med_VQA/data/raw/ImageCLEF/test_2...
